In [119]:
import pybamm
import numpy as np
from SALib.sample import saltelli
from SALib.analyze import sobol
import matplotlib.pyplot as plt
import pandas as pd

In [120]:
param = pybamm.ParameterValues("OKane2022")
print(param)

{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.2594,
 'Cell cooling surface area [m2]': 0.00531,
 'Cell thermal expansion coefficient [m.K-1]': 1.1e-06,
 'Cell volume [m3]': 2.42e-05,
 'Contact resistance [Ohm]': 0,
 'Current function [A]': 5.0,
 'Dead lithium decay constant [s-1]': 1e-06,
 'Dead lithium decay rate [s-1]': <function SEI_limited_dead_lithium_OKane2022 at 0x13cf26b60>,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Electrode height [m]': 0.065,
 'Electrode width [m]': 1.58,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Nyman2008_arrhenius at 0x13cf27240>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Nyman2008_arrhenius at 0x13cf271a0>,
 'Electron charge [C]': 1.602176634e-19,
 'Exchange-current density for plating [A.m-2]': <function plating_exchange_curr

In [121]:
param.search("Maximum")

Maximum concentration in negative electrode [mol.m-3]	33133.0
Maximum concentration in positive electrode [mol.m-3]	63104.0


In [122]:
model = pybamm.lithium_ion.DFN()
param_del = model.default_parameter_values
print(param_del)

{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.4,
 'Cell cooling surface area [m2]': 0.0569,
 'Cell volume [m3]': 7.8e-06,
 'Contact resistance [Ohm]': 0,
 'Current function [A]': 0.680616,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Edge heat transfer coefficient [W.m-2.K-1]': 0.3,
 'Electrode height [m]': 0.137,
 'Electrode width [m]': 0.207,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Capiglia1999 at 0x1435b7380>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Capiglia1999 at 0x1435b4040>,
 'Electron charge [C]': 1.602176634e-19,
 'Faraday constant [C.mol-1]': 96485.33212,
 'Ideal gas constant [J.K-1.mol-1]': 8.314462618,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentration in negative electrode [mol.m-3]': 19986.609595075,
 'Initial 

In [123]:
param_del.search("max")

Maximum concentration in negative electrode [mol.m-3]	24983.2619938437
Maximum concentration in positive electrode [mol.m-3]	51217.9257309275


In [124]:
# Define the model
def battery_model(parameters):
    # Creating a DFN model
    model = pybamm.lithium_ion.DFN()
    
    # Loading the default parameter set
    param = pybamm.ParameterValues("OKane2022")
    
    # Updating parameters
    param.update({
        "Negative electrode thickness [m]": parameters[0],
        "Positive electrode thickness [m]": parameters[1],
        "Separator thickness [m]": parameters[2],
        "Negative electrode active material volume fraction": parameters[3],
        "Positive electrode active material volume fraction": parameters[4],
        "Negative particle radius [m]": parameters[5],
        "Positive particle radius [m]": parameters[6],
        "Initial concentration in electrolyte [mol.m-3]": parameters[7],
        "Negative electrode conductivity [S.m-1]": parameters[8],
        "Positive electrode conductivity [S.m-1]": parameters[9],
        "Negative electrode Bruggeman coefficient (electrode)": parameters[10],
        "Positive electrode Bruggeman coefficient (electrode)": parameters[11],
        "Separator Bruggeman coefficient (electrolyte)": parameters[12],
        "Maximum concentration in negative electrode [mol.m-3]": parameters[13],
        "Maximum concentration in positive electrode [mol.m-3]": parameters[14]
    })
    
    # Creating a Casadi solver and setting options
    solver = pybamm.CasadiSolver(
        mode="safe", 
        atol=1e-1, 
        rtol=1e-1, 
        dt_max=0.01,  # Decrease the maximum time step
        max_step_decrease_count=2000  # Increase in step size decrease in frequency
    )
    
    # Creating a Simulation
    sim = pybamm.Simulation(model, parameter_values=param, solver=solver)
    
    # running simulation
    try:
        sim.solve([0, 450])  # Reduced simulation time range of 450 seconds
    except pybamm.SolverError as e:
        print(f"Solver failed for parameters: {parameters} with error: {e}")
        return np.nan  # Returns NaN to continue processing other combinations of parameters
    
    # Extract simulation results
    solution = sim.solution
    voltage = solution["Terminal voltage [V]"].entries
    
    # Returns the voltage at the end of the simulation
    return voltage[-1]


In [125]:
# Defining the parameter space
problem = {
    'num_vars': 15,
    'names': ['Negative electrode thickness [m]', 
              'Positive electrode thickness [m]', 
              'Separator thickness [m]',
              'Negative electrode active material volume fraction',
              'Positive electrode active material volume fraction',
              'Negative particle radius [m]',
              'Positive particle radius [m]',
              'Initial concentration in electrolyte [mol.m-3]',
              'Negative electrode conductivity [S.m-1]',
              'Positive electrode conductivity [S.m-1]',
              'Negative electrode Bruggeman coefficient (electrode)',
              'Positive electrode Bruggeman coefficient (electrode)',
              'Separator Bruggeman coefficient (electrolyte)',
              'Maximum concentration in negative electrode [mol.m-3]',
              'Maximum concentration in positive electrode [mol.m-3]'
              ],
    'bounds': [[5e-5, 8.6e-5],  # Narrowing down the parameters  8.52e-05
               [5e-5, 7.6e-5],  #7.56e-05
               [1e-5, 2e-5],    #1.2e-05
               [0.7, 0.8],      #0.75
               [0.5, 0.7],      #0.665
               [4e-6, 6e-6],    #5.86e-06
               [4e-6, 6e-6],    #5.22e-06
               [0.9e3, 1.2e3],     # Initial concentration in electrolyte 1000
               [1e2, 3e2],      #215.0
               [0, 10],         #0.18
               [1.3, 1.7],      #1.5
               [1.3, 1.7],      #1.5
               [1.3, 1.7],      #1.5
               [2.3e4, 3.4e4],  #33133.0
               [5.0e4, 6.5e4]   #63104.0
               ]       
}

In [126]:
# Generate a sample
"""原样本生产为1024，现缩减为256"""
param_values = saltelli.sample(problem, 128)

In [127]:
print(param_values)

[[5.04218750e-05 5.86328125e-05 1.29296875e-05 ... 1.62656250e+00
  3.26679688e+04 5.04101562e+04]
 [7.74218750e-05 5.86328125e-05 1.29296875e-05 ... 1.62656250e+00
  3.26679688e+04 5.04101562e+04]
 [5.04218750e-05 6.47265625e-05 1.29296875e-05 ... 1.62656250e+00
  3.26679688e+04 5.04101562e+04]
 ...
 [6.87031250e-05 6.04609375e-05 1.42578125e-05 ... 1.58281250e+00
  3.00039062e+04 5.01757812e+04]
 [6.87031250e-05 6.04609375e-05 1.42578125e-05 ... 1.58281250e+00
  3.36132812e+04 5.91992188e+04]
 [6.87031250e-05 6.04609375e-05 1.42578125e-05 ... 1.58281250e+00
  3.36132812e+04 5.01757812e+04]]


In [128]:
# Run the model and get the results
Y = []

for i, params in enumerate(param_values):
    try:
        result = battery_model(params)
        Y.append(result)
    except Exception as e:
        print(f"Error for sample {i} with parameters {params}: {e}")
        Y.append(np.nan)  # Returns NaN to continue processing other combinations of parameters

# Converting Y to a NumPy array and filtering NaN values
Y = np.array(Y)
valid_indices = ~np.isnan(Y)
Y_valid = Y[valid_indices]
param_values_valid = param_values[valid_indices]

CasADi - 2024-06-28 10:06:06 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:06:06 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:06:06 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:06:06 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:06:06 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]


Solver failed for parameters: [5.04218750e-05 5.86328125e-05 1.29296875e-05 7.19140625e-01
 6.80468750e-01 5.35156250e-06 5.72656250e-06 9.73828125e+02
 1.21093750e+02 8.16406250e+00 1.59531250e+00 1.59531250e+00
 1.62656250e+00 2.85429688e+04 5.04101562e+04] with error: Events ['Maximum voltage [V]'] are non-positive at initial conditions


CasADi - 2024-06-28 10:06:07 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:06:07 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:06:07 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:06:07 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:06:07 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]


Solver failed for parameters: [5.04218750e-05 6.47265625e-05 1.35546875e-05 7.08203125e-01
 6.11718750e-01 5.71093750e-06 4.11718750e-06 1.16601562e+03
 2.86718750e+02 6.67968750e+00 1.57343750e+00 1.33593750e+00
 1.58593750e+00 2.85429688e+04 5.47460938e+04] with error: Events ['Maximum voltage [V]'] are non-positive at initial conditions


CasADi - 2024-06-28 10:06:08 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:06:08 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:06:08 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:06:08 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:06:08 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]


...

CasADi - 2024-06-28 10:16:40 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:16:40 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:16:40 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:16:40 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:16:40 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]


Solver failed for parameters: [7.62968750e-05 5.09140625e-05 1.57421875e-05 7.67578125e-01
 5.67968750e-01 5.64843750e-06 5.67968750e-06 1.13789062e+03
 1.05468750e+02 8.24218750e+00 1.33593750e+00 1.37343750e+00
 1.64843750e+00 2.88867188e+04 5.08789062e+04] with error: Events ['Maximum voltage [V]'] are non-positive at initial conditions


CasADi - 2024-06-28 10:16:41 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:16:41 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:16:41 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:16:41 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]
CasADi - 2024-06-28 10:16:41 WARNING("roots:g failed: Inf detected for output x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:326]


Solver failed for parameters: [7.62968750e-05 5.09140625e-05 1.57421875e-05 7.67578125e-01
 5.67968750e-01 5.64843750e-06 5.67968750e-06 1.13789062e+03
 1.05468750e+02 8.24218750e+00 1.33593750e+00 1.37343750e+00
 1.64843750e+00 2.88867188e+04 5.70898438e+04] with error: Events ['Maximum voltage [V]'] are non-positive at initial conditions


KeyboardInterrupt: 

In [ ]:
# 运行模型并获取结果
Y = []

for i, params in enumerate(param_values):
    try:
        result = battery_model(params)
        Y.append(result)
    except Exception as e:
        print(f"Error for sample {i} with parameters {params}: {e}")
        Y.append(np.nan)  # 返回NaN以继续处理其他参数组合

# 转换Y为NumPy数组并过滤NaN值
Y = np.array(Y)
valid_indices = ~np.isnan(Y)
Y_valid = Y[valid_indices]
param_values_valid = param_values[valid_indices]

In [ ]:
# Perform sensitivity analyses
Si = sobol.analyze(problem, Y_valid, print_to_console=True)

In [117]:
# Print results
print("Sobol Sensitivity Indices")
print("S1:", Si['S1'])
print("ST:", Si['ST'])
print("S2:", Si['S2'])

AttributeError: 'list' object has no attribute 'size'